In [1]:
import pandas as pd
from io import StringIO
import io
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from keras.metrics import FalseNegatives, FalsePositives
from keras.optimizers import Adam
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pickle

In [2]:
def predict_classes(model, x, batch_size=32, verbose=1):
    '''Generate class predictions for the input samples
    batch by batch.
    # Arguments
        x: input data, as a Numpy array or list of Numpy arrays
            (if the model has multiple inputs).
        batch_size: integer.
        verbose: verbosity mode, 0 or 1.
    # Returns
        A numpy array of class predictions.
    '''
    proba = model.predict(x, batch_size=batch_size, verbose=verbose)
    if proba.shape[-1] > 1:
        return proba.argmax(axis=-1)
    else:
        return (proba > 0.5).astype('int32')

In [3]:
rate = 0.3
base_path = "/content/drive/MyDrive/Okul/İTÜ/Bilgisayar Mühendisliği Yüksek Lisans/Yüksek Lisans/YL Tez/Development/CIC-DDoS2019"
X_train = np.genfromtxt(f"{base_path}/Sampled_data/30-sampled-x-train.csv", delimiter=',')
y_train = np.genfromtxt(f"{base_path}/Sampled_data/30-sampled-y-train.csv", delimiter=',')
X_test = np.genfromtxt(f"{base_path}/Sampled_data/30-sampled-x-test.csv", delimiter=',')
y_test = np.genfromtxt(f"{base_path}/Sampled_data/30-sampled-y-test.csv", delimiter=',')

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = y_train.astype("int64")
y_test = y_test.astype("int64")

X_arr = np.asarray(X_train).astype(np.float32)
X_arr = np.resize(X_train,(X_train.shape[0],1,X_train.shape[1]))

y_arr = np.asarray(y_train).astype(np.float32)

In [ ]:
# Model 1
X_columns=[x for x in X.columns if x!='label']
features=len(X_columns)

model_1 = Sequential()
model_1.add(LSTM(100, dropout=0.5, activation='tanh', return_sequences=True, input_shape=(1, features)))
model_1.add(LSTM(49, dropout=0.5, activation='tanh'))
model_1.add(Dense(1, activation='sigmoid'))
history=model_1.compile(optimizer=Adam(lr=0.01), loss='binary_crossentropy', metrics=['accuracy'])

callback = EarlyStopping(monitor='accuracy', patience=3)
model_1.fit(X_arr,y_arr,batch_size=10, epochs=1000, callbacks=[callback])
pickle.dump(model_1, open(f"/content/drive/MyDrive/Okul/İTÜ/Bilgisayar Mühendisliği Yüksek Lisans/Yüksek Lisans/YL Tez/Development/CIC-DDoS2019/Model Development/LSTM-all-data-binary-model-1/{rate}/model.pkl", 'wb'))

Epoch 1/1000


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


5266/5266 [==============================] - 34s 6ms/step - loss: 0.0705 - accuracy: 0.9734
Epoch 2/1000
5266/5266 [==============================] - 38s 7ms/step - loss: 0.0530 - accuracy: 0.9798
Epoch 3/1000
5266/5266 [==============================] - 29s 6ms/step - loss: 0.0495 - accuracy: 0.9815
Epoch 4/1000
5266/5266 [==============================] - 29s 5ms/step - loss: 0.0483 - accuracy: 0.9817
Epoch 5/1000
5266/5266 [==============================] - 30s 6ms/step - loss: 0.0482 - accuracy: 0.9823
Epoch 6/1000
5266/5266 [==============================] - 32s 6ms/step - loss: 0.0454 - accuracy: 0.9836
Epoch 7/1000
5266/5266 [==============================] - 27s 5ms/step - loss: 0.0452 - accuracy: 0.9827
Epoch 8/1000
5266/5266 [==============================] - 28s 5ms/step - loss: 0.0430 - accuracy: 0.9834
Epoch 9/1000
5266/5266 [==============================] - 28s 5ms/step - loss: 0.0441 - accuracy: 0.9831


In [ ]:
model_1 = pickle.load(open(f"/content/drive/MyDrive/Okul/İTÜ/Bilgisayar Mühendisliği Yüksek Lisans/Yüksek Lisans/YL Tez/Development/CIC-DDoS2019/Model Development/LSTM-all-data-binary-model-1/{rate}/model.pkl", 'rb'))
X_t = np.asarray(X_test).astype(np.float32)
X_t = np.resize(X_t,(X_t.shape[0],1,X_t.shape[1]))
y_pred = predict_classes(model_1,X_t)
print("Model 1")
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"False Positive rate: {1 - (cm[0][1] / len(y_test))}")
print(f"False Negative rate: {1 - (cm[1][0] / len(y_test))}")

811/811 [==============================] - 3s 3ms/step
Model 1
[[18554    50]
 [   46  7285]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18604
           1       0.99      0.99      0.99      7331

    accuracy                           1.00     25935
   macro avg       1.00      1.00      1.00     25935
weighted avg       1.00      1.00      1.00     25935

Accuracy: 0.9962984384037016
False Positive rate: 0.9980721033352612
False Negative rate: 0.9982263350684404


In [5]:
# Model 2
model_2 = Sequential()
model_2.add(LSTM(100, activation='tanh', return_sequences=True, input_shape=(1, X_train.shape[1])))
model_2.add(LSTM(49, activation='tanh'))
model_2.add(Dense(1, activation='sigmoid'))
history=model_2.compile(optimizer="adam", loss='binary_crossentropy', metrics=['FalseNegatives'])

callback = EarlyStopping(monitor='false_negatives', patience=50)
model_2.fit(X_arr,y_arr,batch_size=10, epochs=500, callbacks=[callback])

Epoch 1/500
5266/5266 [==============================] - 32s 5ms/step - loss: 0.0168 - false_negatives: 48.0000
Epoch 2/500
5266/5266 [==============================] - 29s 5ms/step - loss: 0.0078 - false_negatives: 15.0000
Epoch 3/500
5266/5266 [==============================] - 30s 6ms/step - loss: 0.0074 - false_negatives: 12.0000
Epoch 4/500
5266/5266 [==============================] - 27s 5ms/step - loss: 0.0064 - false_negatives: 7.0000
Epoch 5/500
5266/5266 [==============================] - 26s 5ms/step - loss: 0.0064 - false_negatives: 16.0000
Epoch 6/500
5266/5266 [==============================] - 26s 5ms/step - loss: 0.0061 - false_negatives: 19.0000
Epoch 7/500
5266/5266 [==============================] - 27s 5ms/step - loss: 0.0052 - false_negatives: 15.0000
Epoch 8/500
5266/5266 [==============================] - 26s 5ms/step - loss: 0.0046 - false_negatives: 29.0000
Epoch 9/500
5266/5266 [==============================] - 26s 5ms/step - loss: 0.0031 - false_negatives: 1

In [6]:
model_2 = pickle.load(open(f"/content/drive/MyDrive/Okul/İTÜ/Bilgisayar Mühendisliği Yüksek Lisans/Yüksek Lisans/YL Tez/Development/CIC-DDoS2019/Model Development/LSTM-all-data-binary-model-2/{rate}/model.pkl", 'rb'))
X_t = np.asarray(X_test).astype(np.float32)
X_t = np.resize(X_t,(X_t.shape[0],1,X_t.shape[1]))
y_pred = predict_classes(model_2,X_t)
print("Model 2")
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"False Positive rate: {1 - (cm[0][1] / len(y_test))}")
print(f"False Negative rate: {1 - (cm[1][0] / len(y_test))}")

811/811 [==============================] - 3s 2ms/step
Model 2
[[18602     2]
 [    8  7323]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18604
           1       1.00      1.00      1.00      7331

    accuracy                           1.00     25935
   macro avg       1.00      1.00      1.00     25935
weighted avg       1.00      1.00      1.00     25935

Accuracy: 0.9996144206670522
False Positive rate: 0.9999228841334105
False Negative rate: 0.9996915365336418


In [7]:
np.savetxt(f"{base_path}/Model Development/lstm-y-pred.csv", y_pred, delimiter=",")

In [ ]:
# Model 3
X_columns=[x for x in X.columns if x!='label']
features=len(X_columns)

model_3 = Sequential()
model_3.add(LSTM(100, activation='tanh', return_sequences=True, input_shape=(1, features)))
model_3.add(LSTM(49, activation='tanh'))
model_3.add(Dense(1, activation='sigmoid'))
history=model_3.compile(optimizer="adam", loss='binary_crossentropy', metrics=['FalsePositives'])

callback = EarlyStopping(monitor='false_positives', patience=50)
model_3.fit(X_arr,y_arr,batch_size=len(X_arr), epochs=500, callbacks=[callback])

Epoch 1/500
1/1 [==============================] - 6s 6s/step - loss: 0.6881 - false_positives: 16815.0000
Epoch 2/500
1/1 [==============================] - 1s 982ms/step - loss: 0.6819 - false_positives: 2533.0000
Epoch 3/500
1/1 [==============================] - 1s 910ms/step - loss: 0.6756 - false_positives: 1009.0000
Epoch 4/500
1/1 [==============================] - 1s 919ms/step - loss: 0.6693 - false_positives: 593.0000
Epoch 5/500
1/1 [==============================] - 1s 924ms/step - loss: 0.6629 - false_positives: 372.0000
Epoch 6/500
1/1 [==============================] - 1s 920ms/step - loss: 0.6564 - false_positives: 249.0000
Epoch 7/500
1/1 [==============================] - 1s 913ms/step - loss: 0.6497 - false_positives: 184.0000
Epoch 8/500
1/1 [==============================] - 1s 916ms/step - loss: 0.6427 - false_positives: 208.0000
Epoch 9/500
1/1 [==============================] - 1s 908ms/step - loss: 0.6355 - false_positives: 209.0000
Epoch 10/500
1/1 [=========

In [ ]:
y_pred_3 = predict_classes(model_3,X_t)
print("Model 3")
print(confusion_matrix(y_test, y_pred_3))
print(classification_report(y_test, y_pred_3))

811/811 [==============================] - 2s 2ms/step
Model 3
[[18489   115]
 [  194  7137]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18604
           1       0.98      0.97      0.98      7331

    accuracy                           0.99     25935
   macro avg       0.99      0.98      0.99     25935
weighted avg       0.99      0.99      0.99     25935

